In [ ]:
from pathlib import Path
from typing import cast

import plotly.graph_objects as go
import polars as pl
from plotly.subplots import make_subplots

In [56]:
INITIAL_MV = 100_000_000
HAIRCUT = 0.02
CAPITAL = INITIAL_MV * 2 * HAIRCUT

In [57]:
def get_data(
    source: str | Path,
) -> pl.DataFrame:
    return cast(
        pl.DataFrame,
        pl.read_excel(
            source=source,
        ).with_columns(pl.col("date").str.to_date()),
    )

In [58]:
data = get_data(source="data/famabliss_strips_2025-11-28.xlsx")
nov_data = data.filter(
    (pl.col("date").dt.month() == 11)
    & (pl.col("date").dt.year().is_between(2020, 2025))
).sort("date")

print(nov_data)

shape: (6, 6)
┌────────────┬──────────┬──────────┬──────────┬──────────┬──────────┐
│ date       ┆ 1        ┆ 2        ┆ 3        ┆ 4        ┆ 5        │
│ ---        ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      │
│ date       ┆ f64      ┆ f64      ┆ f64      ┆ f64      ┆ f64      │
╞════════════╪══════════╪══════════╪══════════╪══════════╪══════════╡
│ 2020-11-30 ┆ 0.998839 ┆ 0.997047 ┆ 0.994464 ┆ 0.988809 ┆ 0.981515 │
│ 2021-11-30 ┆ 0.997523 ┆ 0.988614 ┆ 0.974796 ┆ 0.958322 ┆ 0.943772 │
│ 2022-11-30 ┆ 0.954375 ┆ 0.91822  ┆ 0.887608 ┆ 0.857356 ┆ 0.830593 │
│ 2023-11-30 ┆ 0.95108  ┆ 0.911951 ┆ 0.876957 ┆ 0.842191 ┆ 0.809378 │
│ 2024-11-29 ┆ 0.959187 ┆ 0.920923 ┆ 0.88544  ┆ 0.850418 ┆ 0.817795 │
│ 2025-11-28 ┆ 0.964936 ┆ 0.932866 ┆ 0.900919 ┆ 0.86824  ┆ 0.836057 │
└────────────┴──────────┴──────────┴──────────┴──────────┴──────────┘


# The Carry Trade

## 1.1

In [59]:
z_2020 = (
    nov_data.filter(pl.col("date").dt.year() == 2020).select(pl.exclude("date")).row(0)
)
long_face = INITIAL_MV / z_2020[4]
print(f"""Initial Setup (Nov 2020):
    Long 5yr bond: $100M MV → Face = ${long_face:,.2f}
    Short 1yr bond: $100M MV → Face = ${INITIAL_MV / z_2020[0]:,.2f}
    Capital required (4% haircut): ${CAPITAL:,.0f}""")

Initial Setup (Nov 2020):
    Long 5yr bond: $100M MV → Face = $101,883,281.16
    Short 1yr bond: $100M MV → Face = $100,116,227.51
    Capital required (4% haircut): $4,000,000


In [60]:
positions = (
    nov_data.with_columns(
        year=pl.col("date").dt.year(),
        years_remaining=(5 - (pl.col("date").dt.year() - 2020)).cast(pl.Int64),
    )
    .filter(pl.col("year") <= 2025)
    .with_columns(
        long_price=pl.when(pl.col("years_remaining") == 5)
        .then(pl.col("5"))
        .when(pl.col("years_remaining") == 4)
        .then(pl.col("4"))
        .when(pl.col("years_remaining") == 3)
        .then(pl.col("3"))
        .when(pl.col("years_remaining") == 2)
        .then(pl.col("2"))
        .when(pl.col("years_remaining") == 1)
        .then(pl.col("1"))
        .otherwise(1.0),
    )
    .with_columns(
        long_mv=pl.lit(long_face) * pl.col("long_price"),
        short_1yr_price=pl.col("1"),
    )
)

cash_flows = []
short_face_prev = INITIAL_MV / z_2020[0]

for year in range(2021, 2025):
    row = nov_data.filter(pl.col("date").dt.year() == year).row(0, named=True)

    close_short_cost = short_face_prev

    if year < 2024:
        new_short_proceeds = INITIAL_MV
        short_face_prev = INITIAL_MV / row["1"]
    else:
        new_short_proceeds = 0
        short_face_prev = 0

    net_cf = new_short_proceeds - close_short_cost
    cash_flows.append({"year": year, "cash_flow": net_cf})

cf_df = pl.DataFrame(cash_flows)

pnl = (
    positions.join(cf_df, left_on="year", right_on="year", how="left")
    .with_columns(pl.col("cash_flow").fill_null(0))
    .with_columns(cumulative_cf=pl.col("cash_flow").cum_sum())
    .with_columns(
        short_mv=pl.when(pl.col("year") < 2024).then(pl.lit(-INITIAL_MV)).otherwise(0),
    )
    .with_columns(
        net_position=pl.col("long_mv") + pl.col("short_mv") + pl.col("cumulative_cf"),
        pnl_ytd=pl.col("long_mv") + pl.col("short_mv") + pl.col("cumulative_cf"),
    )
    .select(
        [
            "date",
            "year",
            "years_remaining",
            "long_mv",
            "short_mv",
            "cash_flow",
            "cumulative_cf",
            "pnl_ytd",
        ]
    )
)

print("\nYear-by-Year P&L:")
print(
    pnl.select(["year", "long_mv", "short_mv", "cash_flow", "cumulative_cf", "pnl_ytd"])
)


Year-by-Year P&L:
shape: (6, 6)
┌──────┬──────────┬────────────┬────────────────┬────────────────┬───────────┐
│ year ┆ long_mv  ┆ short_mv   ┆ cash_flow      ┆ cumulative_cf  ┆ pnl_ytd   │
│ ---  ┆ ---      ┆ ---        ┆ ---            ┆ ---            ┆ ---       │
│ i32  ┆ f64      ┆ i32        ┆ f64            ┆ f64            ┆ f64       │
╞══════╪══════════╪════════════╪════════════════╪════════════════╪═══════════╡
│ 2020 ┆ 1e8      ┆ -100000000 ┆ 0.0            ┆ 0.0            ┆ 0.0       │
│ 2021 ┆ 9.7637e7 ┆ -100000000 ┆ -116227.507512 ┆ -116227.507512 ┆ -2.4792e6 │
│ 2022 ┆ 9.0432e7 ┆ -100000000 ┆ -248347.304493 ┆ -364574.812004 ┆ -9.9322e6 │
│ 2023 ┆ 9.2913e7 ┆ -100000000 ┆ -4.7806e6      ┆ -5.1452e6      ┆ -1.2233e7 │
│ 2024 ┆ 9.7725e7 ┆ 0          ┆ -1.0514e8      ┆ -1.1029e8      ┆ -1.2564e7 │
│ 2025 ┆ 1.0188e8 ┆ 0          ┆ 0.0            ┆ -1.1029e8      ┆ -8.4056e6 │
└──────┴──────────┴────────────┴────────────────┴────────────────┴───────────┘


In [61]:
final_row = pnl.filter(pl.col("year") == 2025)
final_value = long_face
total_cash_outflow = cf_df.select(pl.col("cash_flow").sum()).item()
total_profit = final_value + total_cash_outflow
net_return = total_profit / CAPITAL

print(f"""Final Results (Nov 2025):
    Long bond matures, receive face: ${final_value:,.2f}
    Total cash for rolls (negative = outflow): ${total_cash_outflow:,.2f}
    Net profit (face + roll costs): ${total_profit:,.2f}
    Initial capital (margin): ${CAPITAL:,.0f}
    Total Return on ${CAPITAL / 1e6:.0f}M capital: {net_return * 100:.2f}%""")

Final Results (Nov 2025):
    Long bond matures, receive face: $101,883,281.16
    Total cash for rolls (negative = outflow): $-110,288,833.17
    Net profit (face + roll costs): $-8,405,552.01
    Initial capital (margin): $4,000,000
    Total Return on $4M capital: -210.14%


## 1.2

In [62]:
forward_1yr = (
    nov_data.filter(pl.col("date").dt.year() == 2020)
    .unpivot(index="date", variable_name="maturity", value_name="price")
    .with_columns(pl.col("maturity").cast(pl.Int64))
    .sort("maturity")
    .with_columns(
        prev_price=pl.col("price").shift(1),
    )
    .filter(pl.col("maturity") > 1)
    .with_columns(
        forward_price=pl.col("price") / pl.col("prev_price"),
        forward_year=pl.col("maturity") + 2019,
    )
    .select(["forward_year", "forward_price"])
)

initial_1yr = pl.DataFrame({"forward_year": [2020], "forward_price": [z_2020[0]]})
hypothetical_1yr = pl.concat([initial_1yr, forward_1yr]).sort("forward_year")

print("\nImplied 1-year forward rates from Nov 2020 yield curve:")
print(
    hypothetical_1yr.with_columns(
        implied_rate=(-pl.col("forward_price").log()).round(4)
    )
)


Implied 1-year forward rates from Nov 2020 yield curve:
shape: (5, 3)
┌──────────────┬───────────────┬──────────────┐
│ forward_year ┆ forward_price ┆ implied_rate │
│ ---          ┆ ---           ┆ ---          │
│ i64          ┆ f64           ┆ f64          │
╞══════════════╪═══════════════╪══════════════╡
│ 2020         ┆ 0.998839      ┆ 0.0012       │
│ 2021         ┆ 0.998206      ┆ 0.0018       │
│ 2022         ┆ 0.997409      ┆ 0.0026       │
│ 2023         ┆ 0.994314      ┆ 0.0057       │
│ 2024         ┆ 0.992624      ┆ 0.0074       │
└──────────────┴───────────────┴──────────────┘


In [63]:
hypo_cash_flows = []
short_face_prev = INITIAL_MV / z_2020[0]

for row in hypothetical_1yr.iter_rows(named=True):
    year = row["forward_year"]
    if year == 2020:
        continue
    if year > 2024:
        break

    close_short_cost = short_face_prev

    if year < 2024:
        fwd_price = (
            hypothetical_1yr.filter(pl.col("forward_year") == year)
            .select("forward_price")
            .item()
        )
        new_short_proceeds = INITIAL_MV
        short_face_prev = INITIAL_MV / fwd_price
    else:
        new_short_proceeds = 0

    net_cf = new_short_proceeds - close_short_cost
    hypo_cash_flows.append({"year": year, "cash_flow": net_cf})

hypo_cf_df = pl.DataFrame(hypo_cash_flows)
hypo_total_cash = hypo_cf_df.select(pl.col("cash_flow").sum()).item()
hypo_total_profit = final_value + hypo_total_cash
hypo_net_return = hypo_total_profit / CAPITAL

print("\nHypothetical Results (if forwards = realized spots):")
print(f"  Long bond matures: ${final_value:,.2f}")
print(f"  Total cash for rolls: ${hypo_total_cash:,.2f}")
print(f"  Net profit: ${hypo_total_profit:,.2f}")
print(f"  Total Return on ${CAPITAL / 1e6:.0f}M capital: {hypo_net_return * 100:.2f}%")


Hypothetical Results (if forwards = realized spots):
  Long bond matures: $101,883,281.16
  Total cash for rolls: $-101,127,603.13
  Net profit: $755,678.03
  Total Return on $4M capital: 18.89%


In [67]:
comparison = (
    cf_df.rename({"cash_flow": "actual_cf"})
    .join(hypo_cf_df.rename({"cash_flow": "hypothetical_cf"}), on="year")
    .with_columns(difference=pl.col("actual_cf") - pl.col("hypothetical_cf"))
)
print("\nCash flow comparison (actual vs hypothetical):")
print(comparison)


Cash flow comparison (actual vs hypothetical):
shape: (4, 4)
┌──────┬────────────────┬─────────────────┬───────────────┐
│ year ┆ actual_cf      ┆ hypothetical_cf ┆ difference    │
│ ---  ┆ ---            ┆ ---             ┆ ---           │
│ i64  ┆ f64            ┆ f64             ┆ f64           │
╞══════╪════════════════╪═════════════════╪═══════════════╡
│ 2021 ┆ -116227.507512 ┆ -116227.507512  ┆ 0.0           │
│ 2022 ┆ -248347.304493 ┆ -179740.925765  ┆ -68606.378727 │
│ 2023 ┆ -4.7806e6      ┆ -259748.339154  ┆ -4.5209e6     │
│ 2024 ┆ -1.0514e8      ┆ -1.0057e8       ┆ -4.5718e6     │
└──────┴────────────────┴─────────────────┴───────────────┘


In [65]:
current_forward = (
    nov_data.filter(pl.col("date").dt.year() == 2025)
    .unpivot(index="date", variable_name="maturity", value_name="price")
    .with_columns(pl.col("maturity").cast(pl.Int64))
    .sort("maturity")
    .with_columns(prev_price=pl.col("price").shift(1))
    .filter(pl.col("maturity") > 1)
    .with_columns(
        forward_price=pl.col("price") / pl.col("prev_price"),
        forward_rate=-((pl.col("price") / pl.col("prev_price")).log()),
    )
)

print("\n2025 Forward rates vs 2020 Forward rates:")

fwd_2020_rates = (
    nov_data.filter(pl.col("date").dt.year() == 2020)
    .unpivot(index="date", variable_name="maturity", value_name="price")
    .with_columns(pl.col("maturity").cast(pl.Int64))
    .sort("maturity")
    .with_columns(prev_price=pl.col("price").shift(1).fill_null(1.0))
    .with_columns(
        forward_rate=-((pl.col("price") / pl.col("prev_price")).log()),
    )
    .select(["maturity", "forward_rate"])
    .rename({"forward_rate": "fwd_rate_2020"})
)

fwd_2025_rates = (
    nov_data.filter(pl.col("date").dt.year() == 2025)
    .unpivot(index="date", variable_name="maturity", value_name="price")
    .with_columns(pl.col("maturity").cast(pl.Int64))
    .sort("maturity")
    .with_columns(prev_price=pl.col("price").shift(1).fill_null(1.0))
    .with_columns(
        forward_rate=-((pl.col("price") / pl.col("prev_price")).log()),
    )
    .select(["maturity", "forward_rate"])
    .rename({"forward_rate": "fwd_rate_2025"})
)

comparison_fwd = fwd_2020_rates.join(fwd_2025_rates, on="maturity")
print(
    comparison_fwd.with_columns(
        (pl.col("fwd_rate_2020") * 100).round(3).alias("2020 (%)"),
        (pl.col("fwd_rate_2025") * 100).round(3).alias("2025 (%)"),
    ).select(["maturity", "2020 (%)", "2025 (%)"])
)


2025 Forward rates vs 2020 Forward rates:
shape: (5, 3)
┌──────────┬──────────┬──────────┐
│ maturity ┆ 2020 (%) ┆ 2025 (%) │
│ ---      ┆ ---      ┆ ---      │
│ i64      ┆ f64      ┆ f64      │
╞══════════╪══════════╪══════════╡
│ 1        ┆ 0.116    ┆ 3.569    │
│ 2        ┆ 0.18     ┆ 3.38     │
│ 3        ┆ 0.259    ┆ 3.485    │
│ 4        ┆ 0.57     ┆ 3.695    │
│ 5        ┆ 0.74     ┆ 3.777    │
└──────────┴──────────┴──────────┘


## 1.3

> Forward rates are BIASED UPWARD predictors of future spot rates, and the term premium is positive.

In 2020, forward rates implied very low future 1yr rates (0.1-0.7%). Realized rates in 2022-2024 were higher (4-5%) due to Fed hikes.

This means the carry trade LOST money because:
1. Long position lost value as rates rose
2. Rolling short positions became expensive (high short-term rates)

Looking forward (2025-2030):
- Current yield curve is less steep (higher short rates already)
- If EH Fact 3 holds, forwards still overestimate future spot rates
- But the gap is smaller now, so carry trade has LESS favorable risk/reward

The 2020-2025 period was particularly bad because:
- Started with near-zero rates (nowhere to go but up)
- Experienced historic rate hiking cycle

In [68]:
fwd_data_2020 = fwd_2020_rates.to_dicts()
fwd_data_2025 = fwd_2025_rates.to_dicts()

fig = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=(
        "Carry Trade P&L (Year-by-Year)",
        "Long Position Value vs Short",
        "Actual vs Hypothetical Cash Flows",
        "Forward Curves: 2020 vs 2025",
    ),
    vertical_spacing=0.12,
    horizontal_spacing=0.08,
)

pnl_data = pnl.to_dicts()
years = [d["year"] for d in pnl_data]
pnl_values = [d["pnl_ytd"] / 1e6 for d in pnl_data]

fig.add_trace(
    go.Bar(
        x=years,
        y=pnl_values,
        marker_color=["#ef4444" if v < 0 else "#22c55e" for v in pnl_values],
    ),
    row=1,
    col=1,
)

long_mv = [d["long_mv"] / 1e6 for d in pnl_data]
short_mv = [-d["short_mv"] / 1e6 for d in pnl_data]

fig.add_trace(
    go.Scatter(x=years, y=long_mv, name="Long MV", line=dict(color="#3b82f6", width=2)),
    row=1,
    col=2,
)
fig.add_trace(
    go.Scatter(
        x=years,
        y=short_mv,
        name="Short MV",
        line=dict(color="#ef4444", width=2, dash="dash"),
    ),
    row=1,
    col=2,
)

comp_data = comparison.to_dicts()
comp_years = [d["year"] for d in comp_data]
actual_cf = [d["actual_cf"] / 1e6 for d in comp_data]
hypo_cf = [d["hypothetical_cf"] / 1e6 for d in comp_data]

fig.add_trace(
    go.Bar(x=comp_years, y=actual_cf, name="Actual", marker_color="#3b82f6"),
    row=2,
    col=1,
)
fig.add_trace(
    go.Bar(x=comp_years, y=hypo_cf, name="Hypothetical", marker_color="#a855f7"),
    row=2,
    col=1,
)

fwd_2020 = hypothetical_1yr.with_columns(
    rate=(-pl.col("forward_price").log())
).to_dicts()

fig.add_trace(
    go.Scatter(
        x=[d["maturity"] for d in fwd_data_2020],
        y=[d["fwd_rate_2020"] * 100 for d in fwd_data_2020],
        name="2020 Forwards",
        line=dict(color="#f59e0b", width=2),
    ),
    row=2,
    col=2,
)
fig.add_trace(
    go.Scatter(
        x=[d["maturity"] for d in fwd_data_2025],
        y=[d["fwd_rate_2025"] * 100 for d in fwd_data_2025],
        name="2025 Forwards",
        line=dict(color="#06b6d4", width=2),
    ),
    row=2,
    col=2,
)

fig.update_yaxes(title_text="P&L ($M)", row=1, col=1)
fig.update_yaxes(title_text="Market Value ($M)", row=1, col=2)
fig.update_yaxes(title_text="Cash Flow ($M)", row=2, col=1)
fig.update_yaxes(title_text="Rate (%)", row=2, col=2)

fig.update_layout(
    template="plotly_white",
    paper_bgcolor="rgba(0,0,0,0)",
    plot_bgcolor="rgba(0,0,0,0)",
    showlegend=True,
    legend=dict(orientation="h", yanchor="bottom", y=-0.15, xanchor="center", x=0.5),
    height=600,
    width=1000,
    margin=dict(l=50, r=50, t=60, b=80),
    font=dict(size=11, color="#101010"),
)

for i in range(1, 3):
    for j in range(1, 3):
        fig.update_xaxes(showgrid=True, gridcolor="rgba(128,128,128,0.1)", row=i, col=j)
        fig.update_yaxes(showgrid=True, gridcolor="rgba(128,128,128,0.1)", row=i, col=j)

fig.show()